# Setup

In [7]:
NEO4J_START_FROM_SCRATCH = True

# NEO4J_WORKDIR = "/var/lib/neo4j"
NEO4J_PORT = 7687
NEO4J_WEBUI_PORT = 7474

NEO4J_INIT_USER = "neo4j"
NEO4J_INIT_PASSWORD = "password"

In [8]:
import os
from pathlib import Path

LOCALHOST_WORKDIR = f"{os.path.join(os.path.relpath(Path.cwd()))}"
DOCKER_MOUNTDIR = os.path.join(LOCALHOST_WORKDIR, "mount")

mount_path = Path(DOCKER_MOUNTDIR)
mount_path.mkdir(parents=True, exist_ok=True)

## Stop neo4j.docker-compose.yml

In [9]:
!docker compose -f neo4j.docker-compose.yml down -v

 Container neo4j-instance  Stopping
 Container neo4j-instance  Stopped
 Container neo4j-instance  Removing
 Container neo4j-instance  Removed
 Network neo4j-compose_default  Removing
 Network neo4j-compose_default  Removed


In [10]:
import shutil

if NEO4J_START_FROM_SCRATCH:
    shutil.rmtree(DOCKER_MOUNTDIR)
    Path(DOCKER_MOUNTDIR).mkdir(parents=True, exist_ok=True)

# Start neo4j.docker-compose.yml

In [11]:
import os
import yaml
from IPython.display import Markdown, display

neo4j_compose_dict = {
    "name": "neo4j-compose",
    "services": {
        "neo4j": {
            "image": "neo4j:ubi9",
            "container_name": "neo4j-instance",
            "volumes": [
                f"{os.path.join(DOCKER_MOUNTDIR,"data")}:/data",
                f"{os.path.join(DOCKER_MOUNTDIR,"neo4j","logs")}:/logs",
                f"{os.path.join(DOCKER_MOUNTDIR,"neo4j","import")}:/var/lib/neo4j/import",
                f"{os.path.join(DOCKER_MOUNTDIR,"plugins")}:/plugins",
            ],
            "environment": {
                "NEO4J_AUTH": f"{NEO4J_INIT_USER}/{NEO4J_INIT_PASSWORD}",
                # NEO4J 5.x Memory Settings (Double underscores represent single underscores in .conf)
                "NEO4J_server_memory_heap_initial__size": "512m",
                "NEO4J_server_memory_heap_max__size": "1G",
                "NEO4J_server_memory_pagecache_size": "512m",
                "NEO4J_dbms_memory_heap_initial_size": "1024m",
                "NEO4J_dbms_memory_heap_max_size": "2G",
                # Plugin Configuration
                "NEO4J_PLUGINS": '["apoc", "graph-data-science"]',
                # Security: Allow APOC, GDS to run procedures
                "NEO4J_dbms_security_procedures_unrestricted": "apoc.*,gds.*",
                "NEO4J_dbms_security_procedures_allowlist": "apoc.*,gds.*",
                "NEO4J_server_config_strict__validation_enabled": "false",
                "NEO4J_server_bolt_enabled": "true",
            },
            "ports": [
                f"{NEO4J_WEBUI_PORT}:{NEO4J_WEBUI_PORT}",
                f"{NEO4J_PORT}:{NEO4J_PORT}",
            ],
            "restart": "unless-stopped",
            "deploy": {"resources": {"limits": {"cpus": "2.0", "memory": "2048M"}}},
        }
    },
}

neo4j_compose_yaml_path = os.path.join(LOCALHOST_WORKDIR, "neo4j.docker-compose.yml")
neo4j_compose_yaml_contents = yaml.dump(
    neo4j_compose_dict, default_flow_style=False, sort_keys=False, indent=4
)
with open(neo4j_compose_yaml_path, "w") as f:
    f.write(neo4j_compose_yaml_contents)

print(f"Successfully created: '{os.path.relpath(neo4j_compose_yaml_path)}'")
display(Markdown(f"```yaml\n{neo4j_compose_yaml_contents}\n```"))

Successfully created: 'neo4j.docker-compose.yml'


```yaml
name: neo4j-compose
services:
    neo4j:
        image: neo4j:ubi9
        container_name: neo4j-instance
        volumes:
        - .\mount\data:/data
        - .\mount\neo4j\logs:/logs
        - .\mount\neo4j\import:/var/lib/neo4j/import
        - .\mount\plugins:/plugins
        environment:
            NEO4J_AUTH: neo4j/password
            NEO4J_server_memory_heap_initial__size: 512m
            NEO4J_server_memory_heap_max__size: 1G
            NEO4J_server_memory_pagecache_size: 512m
            NEO4J_dbms_memory_heap_initial_size: 1024m
            NEO4J_dbms_memory_heap_max_size: 2G
            NEO4J_PLUGINS: '["apoc", "graph-data-science"]'
            NEO4J_dbms_security_procedures_unrestricted: apoc.*,gds.*
            NEO4J_dbms_security_procedures_allowlist: apoc.*,gds.*
            NEO4J_server_config_strict__validation_enabled: 'false'
            NEO4J_server_bolt_enabled: 'true'
        ports:
        - 7474:7474
        - 7687:7687
        restart: unless-stopped
        deploy:
            resources:
                limits:
                    cpus: '2.0'
                    memory: 2048M

```

In [12]:
!docker compose -f neo4j.docker-compose.yml up -d --wait

 Network neo4j-compose_default  Creating
 Network neo4j-compose_default  Created
 Container neo4j-instance  Creating
 Container neo4j-instance  Created
 Container neo4j-instance  Starting
 Container neo4j-instance  Started
 Container neo4j-instance  Waiting
 Container neo4j-instance  Healthy
